# Chapter 7: Optimizing and Tuning Spark Applications
Christoph Windheuser    
May, 2022   
Python examples of chapter 7 (page 173 ff) in the book *Learning Spark*

In [34]:
# Import required python spark libraries
import pyspark
from pyspark.sql.functions import col, expr, when, concat, lit, avg


In [1]:
#create a SparkSession

spark = (SparkSession \
         .builder \
         .enableHiveSupport() \
         .appName("Chapter_7") \
         .getOrCreate())


In [3]:
# Show the content of the environment variable $SPARK_HOME:
!echo $SPARK_HOME

/opt/spark


In [ ]:
# Show all config files
!ls -l $SPARK_HOME/conf

In [ ]:
#Get single Spark configuiration values:
print(spark.conf.get("spark.sql.warehouse.dir"))

In [ ]:
# Get the whole confiuguration context of a Spark Context:
scConf = sc.getConf().getAll()

for l in scConf:
    print (l[0] + ":")
    print (l[1])
    print ()


In [ ]:
# Change single Spark config variables
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

In [ ]:
# Show the Spark SQL-specifdic Spark configs:
spark.sql("SET -v").select("key", "value").show(truncate=False)

## Spark's Web Interface
To see Spark's Web Interface, go the web address: http://127.0.0.1:4040    
The tab *Environment* shows all environment variables. In the web interface, the variables are *read-only*, they cannot be modified.

## Set configuration variables in a Spark program

In [ ]:
# First check, if a configuration variable is modifiable:

# Example:
spark.conf.isModifiable("spark.sql.shuffle.partitions")

In [ ]:
# Get the actual value of the variable:
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
# Set the variable to a new variable and check:
spark.conf.set("spark.sql.shuffle.partitions", 5)
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
# Set it back to the old value:
spark.conf.set("spark.sql.shuffle.partitions", 16)
spark.conf.get("spark.sql.shuffle.partitions")

## Partitions
Page 181 ff.

In [ ]:
# Create a big DataFrame:
numDF = spark.range(1000 * 1000)

In [ ]:
# Get the default number of partitions of this DataFrame
numDF.rdd.getNumPartitions()

In [ ]:
# Now change the number of partitions to another value
numDF = spark.range(1000 * 1000).repartition(32)

In [ ]:
# Check the number of partitions>:
numDF.rdd.getNumPartitions()

## Caching of Data
Page 183 ff.

Create a DataFrame with 10M records.  

The time difference (approx. 10x faster) between *Count and load into cache*' and *Count in cache*
can only be demonstrated when this code is run the first time in the notebook. In consecutive executions the DataFrame is already cached and there is basically no time difference.

In [4]:


import time

start = time.time()
df = spark.range(1 * 10000000).toDF("id")
end = time.time()
print("Step 1 - Create:                    %f seconds" %(end - start))

start = time.time()
df = df.withColumn("square", df.id * df.id)
end = time.time()
print("Step 2 - Add Column:                %f seconds" %(end - start))

start = time.time()
df.cache()
end = time.time()
print("Step 3 - Cache df:                  %f seconds" %(end - start))

start = time.time()
df.count()
end = time.time()
print("Step 4 - Count and load into cache: %f seconds" %(end - start))

start = time.time()
df.count()
end = time.time()
print("Step 5 - Count in cache:            %f seconds" %(end - start))


Step 1 - Create:                    1.442420 seconds
Step 2 - Add Column:                0.086140 seconds
Step 3 - Cache df:                  0.292966 seconds
Step 4 - Count and load into cache: 3.301321 seconds
Step 5 - Count in cache:            0.181523 seconds


### Caching Tables and Views in SQL
It is also possible to cache tables of views:

In [8]:
df.createOrReplaceTempView("dfTable")
spark.sql("CACHE TABLE dfTable")
spark.sql("SELECT count(*) FROM dfTable").show()

+--------+
|count(1)|
+--------+
|10000000|
+--------+



## Persistance of Data
Page 184 ff

Persistance of data is synonymous to caching data, but let you apecify how the data is persisted with the parameter `pyspark.StorageLevel.LEVEL`. 

As we have specified the persistance on disk only, the time difference is much lower compared to the example above (this time approx. 5x faster compared to 12x faster above). Under the link http://127.0.0.1:4040/ you can see that the data is persisted on disk and not on memory for all partitions.

In [7]:
start = time.time()
df2 = spark.range(1 * 10000000).toDF("id")
end = time.time()
print("Step 1 - Create:                    %f seconds" %(end - start))

start = time.time()
df2 = df.withColumn("square", df2.id * df2.id)
end = time.time()
print("Step 2 - Add Column:                %f seconds" %(end - start))

start = time.time()
df2.persist(storageLevel=pyspark.StorageLevel.DISK_ONLY)
end = time.time()
print("Step 3 - Persist df DISK_ONLY:      %f seconds" %(end - start))

start = time.time()
df2.count()
end = time.time()
print("Step 4 - Count and load into cache: %f seconds" %(end - start))

start = time.time()
df2.count()
end = time.time()
print("Step 5 - Count in cache:            %f seconds" %(end - start))


Step 1 - Create:                    0.019116 seconds
Step 2 - Add Column:                0.006613 seconds
Step 3 - Persist df DISK_ONLY:      0.031600 seconds
Step 4 - Count and load into cache: 1.571908 seconds
Step 5 - Count in cache:            0.259982 seconds


## Shuffle Sort Merge Join (SMJ)
Page 189 ff.

In [11]:
from random import randint

# Disable broadcast join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [13]:
# Generate synthetic data for two data frames
states = ['AZ', 'CO', 'CA', 'TX', 'NY', 'MI']
items  = ['SKU-0', 'SKU-1', 'SKU-2', 'SKU-3', 'SKU-4']

In [59]:
userDF = spark.range(1000000).toDF("uid")

In [60]:
userDF.show(n=3)

+---+
|uid|
+---+
|  0|
|  1|
|  2|
+---+
only showing top 3 rows



In [91]:
userDF = (userDF
            .withColumn("login", concat(lit("user_"), expr("uid")))
            .withColumn("email", concat(lit("user_"), expr("uid"), lit("@databricks.com")))
            .withColumn("user_state", lit(states[randint(0, 5)]))          
         )

In [92]:
userDF.show(n=5)

+---+------+--------------------+----------+
|uid| login|               email|user_state|
+---+------+--------------------+----------+
|  0|user_0|user_0@databricks...|        MI|
|  1|user_1|user_1@databricks...|        MI|
|  2|user_2|user_2@databricks...|        MI|
|  3|user_3|user_3@databricks...|        MI|
|  4|user_4|user_4@databricks...|        MI|
+---+------+--------------------+----------+
only showing top 5 rows



In [76]:
orderDF = spark.range(1000000).toDF("transaction_id")

In [77]:
orderDF.show(n=3)

+--------------+
|transaction_id|
+--------------+
|             0|
|             1|
|             2|
+--------------+
only showing top 3 rows



In [93]:
orderDF = (orderDF
            .withColumn("quantity", orderDF.transaction_id)
            .withColumn("users_id", lit(randint(0, 10000)))
            .withColumn("amount",   orderDF.transaction_id * 2.0)
            .withColumn("state",    lit(states[randint(0,5)]))
            .withColumn("items",    lit(items[randint(0,4)]))
          )

In [94]:
orderDF.show(n=5)

+--------------+--------+--------+------+-----+-----+
|transaction_id|quantity|users_id|amount|state|items|
+--------------+--------+--------+------+-----+-----+
|             0|       0|    2851|   0.0|   AZ|SKU-2|
|             1|       1|    2851|   2.0|   AZ|SKU-2|
|             2|       2|    2851|   4.0|   AZ|SKU-2|
|             3|       3|    2851|   6.0|   AZ|SKU-2|
|             4|       4|    2851|   8.0|   AZ|SKU-2|
+--------------+--------+--------+------+-----+-----+
only showing top 5 rows



In [98]:
userOrderDF = orderDF.join(userDF, orderDF.users_id == userDF.uid)

In [99]:
userOrderDF.show()

+--------------+--------+--------+------+-----+-----+----+---------+--------------------+----------+
|transaction_id|quantity|users_id|amount|state|items| uid|    login|               email|user_state|
+--------------+--------+--------+------+-----+-----+----+---------+--------------------+----------+
|             0|       0|    2851|   0.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             1|       1|    2851|   2.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             2|       2|    2851|   4.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             3|       3|    2851|   6.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             4|       4|    2851|   8.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             5|       5|    2851|  10.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|        MI|
|             6|       6|    2851|  12.0|   AZ|SKU-2|2851|user_2851|user_2851@databri...|  